# Yelp Review Wrangling

In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime

import json

import seaborn as sns
sns.set()
# sns.set_style('whitegrid')
# sns.set_context("poster")

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Load Review Data

In [3]:
print('[%s] Loading Review Data...' % datetime.datetime.now().time())

data = pd.DataFrame()
source_data_file = '../source_data/review.json'

reviews_list = []
for line in open(source_data_file, 'r'):
    reviews_list.append(json.loads(line))

print('[%s] creating dataframe...' % datetime.datetime.now().time())    
reviews_df = pd.DataFrame(reviews_list)

print('[%s] data type cleanup...' % datetime.datetime.now().time())    
reviews_df.date        = pd.to_datetime(reviews_df.date)
    
print('[%s] Complete!' % datetime.datetime.now().time())

[02:37:37.333364] Loading Review Data...
[02:40:04.459353] creating dataframe...
[02:46:48.553212] data type cleanup...
[02:49:20.371445] Complete!


In [4]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4736897 entries, 0 to 4736896
Data columns (total 9 columns):
business_id    object
cool           int64
date           datetime64[ns]
funny          int64
review_id      object
stars          int64
text           object
useful         int64
user_id        object
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 325.3+ MB


In [5]:
reviews_df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,uYHaNptLzDLoV_JZ_MuzUA,0,2016-07-12,0,VfBHSwC5Vz_pbFluy07i9Q,5,My girlfriend and I stayed here for 3 nights a...,0,cjpdDjZyprfyDG3RlkVG3w
1,uYHaNptLzDLoV_JZ_MuzUA,0,2016-10-02,0,3zRpneRKDsOPq92tq7ybAA,3,If you need an inexpensive place to stay for a...,0,bjTcT8Ty4cJZhEOEo01FGA
2,uYHaNptLzDLoV_JZ_MuzUA,0,2015-09-17,0,ne5WhI1jUFOcRn-b-gAzHA,3,Mittlerweile gibt es in Edinburgh zwei Ableger...,0,AXgRULmWcME7J6Ix3I--ww
3,uYHaNptLzDLoV_JZ_MuzUA,0,2016-08-21,0,llmdwOgDReucVoWEry61Lw,4,Location is everything and this hotel has it! ...,0,oU2SSOmsp_A8JYI7Z2JJ5w
4,uYHaNptLzDLoV_JZ_MuzUA,0,2013-11-20,0,DuffS87NaSMDmIfluvT83g,5,gute lage im stadtzentrum. shoppingmeile und s...,0,0xtbPEna2Kei11vsU-U2Mw


# Write to File
<p>Split by year and month</p>

In [6]:
for year in sorted(pd.unique(reviews_df.date.dt.year)):
    year_reviews_df = reviews_df[reviews_df.date.dt.year == year]
    
    for month in sorted(pd.unique(year_reviews_df.date.dt.month)):
        month_reviews_df = year_reviews_df[year_reviews_df.date.dt.month == month]
        
        month_reviews_df.reset_index(inplace=True, drop=True)
        month_reviews_df.to_csv('../clean_data/03_reviews/%s_%s_reviews_clean.csv' % (year, str(month).rjust(2, '0')))
    print('FINISHED %s' % year)

FINISHED 2004
FINISHED 2005
FINISHED 2006
FINISHED 2007
FINISHED 2008
FINISHED 2009
FINISHED 2010
FINISHED 2011
FINISHED 2012
FINISHED 2013
FINISHED 2014
FINISHED 2015
FINISHED 2016
FINISHED 2017
